In [67]:
import sys
sys.path.append("C:\\Users\\Shyam\\AppData\\Local\\Programs\\Python\\Python36\\lib\\site-packages")
sys.path.append(r'D:\shyam\graphviz-2.38\release\bin')
import os
os.environ["PATH"] += os.pathsep + r'D:\shyam\graphviz-2.38\release\bin'
import warnings
warnings.filterwarnings('ignore')

In [433]:
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
import numpy as np
import pandas as pd
import graphviz
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import time

In [434]:
def discretizeAttr(attr,bins):
    attr = pd.qcut(attr,bins,duplicates='drop')
    return attr.astype(str)

In [435]:
from sklearn.preprocessing import OneHotEncoder
def oneHotEncodeRFA(attr,absenteeism):
    enc = OneHotEncoder(handle_unknown='ignore')
    list_rfa = []
    for index,row in absenteeism.iterrows():
        list_rfa.append((row['Reason_for_absence'],row['Reason_for_absence']))
    list_rfa_fit = [[0, 0], [1, 1], [2, 2],[3,3],[4,4]]
    enc.fit(list_rfa_fit)
    encoded = enc.transform(list_rfa).toarray()[:, 0:5]
    absenteeism['RFA0'] = encoded[:, [0]]
    absenteeism['RFA1'] = encoded[:, [1]]
    absenteeism['RFA2'] = encoded[:, [2]]
    absenteeism['RFA3'] = encoded[:, [3]]
    absenteeism['RFA4'] = encoded[:, [4]]
    absenteeism_new = absenteeism.drop('Reason_for_absence',1)
    absenteeism_new = absenteeism_new.drop('Absenteeism_time_in_hours',1)
    absenteeism_new['Absenteeism_time_in_hours'] = absenteeism['Absenteeism_time_in_hours']
    return absenteeism_new

In [436]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
class PCAModified:
    predictors = None
    predictors_pca = None
    pca = None
    def __init__(self,predictors):
        self.predictors = predictors
        self.pca = PCA(.95)
    
    def setVariance(var=.95):
        self.pca = PCA(var)
        
    def reduceDimensions(self):
        scaler = StandardScaler()
        scaler.fit(predictors)
        self.predictors_pca = scaler.transform(self.predictors)
        self.pca.fit(self.predictors_pca)
        self.predictors_pca = self.pca.transform(self.predictors_pca)
        return self.predictors_pca

In [437]:
class DecisionTreeModified:
    confusion_matrix = None
    precision = None
    recall = None
    roc_auc = None
    predictors = None
    response = None
    classifier = None
    fit = None
    cv_scores = None
    cv_folds = 10
    start_time = None
    summary = {}
    
    def __init__(self,predictors,response):
        self.predictors = predictors
        self.response = response
    
    def getSummary(self):
        self.summary['fit'] = self.fit
        self.summary['accuracy'] = self.cv_scores.mean()
        self.summary['confusion_matrix'] = self.confusion_matrix
        self.summary['precision'] = self.precision
        self.summary['recall'] = self.recall
        self.summary['roc_auc'] = self.roc_auc
        self.summary['leaf_nodes'],self.summary['non_leaf_nodes'],self.summary['size'] = self.getNumberOfLeafNodes()
        load_time = time.time() - self.start_time
        self.summary['time_taken'] = "%.4s seconds" % (load_time)
        return self.summary
    
    def getNumberOfLeafNodes(self):
        tree = self.fit.tree_
        children_left = tree.children_left
        children_right = tree.children_right
        n_nodes = tree.node_count
        node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
        is_leaves = np.zeros(shape=n_nodes, dtype=bool)
        stack = [(0, -1)]
        while len(stack) > 0:
            node_id, parent_depth = stack.pop()
            node_depth[node_id] = parent_depth + 1
            if (children_left[node_id] != children_right[node_id]):
                stack.append((children_left[node_id], parent_depth + 1))
                stack.append((children_right[node_id], parent_depth + 1))
            else:
                is_leaves[node_id] = True
        return len(is_leaves[is_leaves==True]),len(is_leaves[is_leaves!=True]),n_nodes
    
    def calculateROCArea(self,y_prediction):
        n_classes = len(self.fit.classes_)
        for i in range(n_classes):
            pos_label_ = self.fit.classes_[i]
            fpr, tpr, _ = roc_curve(response,y_prediction,pos_label_)
            roc_auc = auc(fpr, tpr)
        return roc_auc/n_classes
            
    def trainClassifier(self,parameters={},labels=["'\\'(-inf-1.5]\\''","'\\'(1.5-2.5]\\''","'\\'(2.5-3.5]\\''","'\\'(3.5-7.5]\\''","'\\'(7.5-12]\\''","'\\'(12-inf)\\''"]):
        self.start_time = time.time()
        self.classifier = tree.DecisionTreeClassifier(**parameters)
        self.fit = self.classifier.fit(self.predictors,self.response)
        self.cv_scores = cross_val_score(estimator=self.classifier, X=self.predictors, y=self.response, cv=self.cv_folds)
        y_prediction = cross_val_predict(estimator=self.classifier, X=self.predictors, y=self.response, cv=self.cv_folds)
        self.confusion_matrix = confusion_matrix(response, y_prediction,labels)
        self.precision = precision_score(response, y_prediction, average='macro')
        self.recall = recall_score(response, y_prediction, average='macro')
        #self.roc_auc = self.calculateROCArea(y_prediction)
        return self.getSummary()
    
    def saveTree(self,filename):
        dot_data = tree.export_graphviz(self.classifier, feature_names = self.predictors.columns, out_file=None) 
        graph = graphviz.Source(dot_data) 
        graph.render(filename)       

# Experiments with no pre-processing

In [456]:
absenteeism = pd.read_csv(r'D:\WPI\DS-Coursework\CS548-KDD\Homework\HW2\dataset\Absenteeism_at_work_AAA\Absenteeism_python.csv')
predictors = absenteeism.iloc[:,0:17]
response = absenteeism.iloc[:,17]
dt_initial = DecisionTreeModified(predictors,response)

In [460]:
summary = dt_initial.trainClassifier({'criterion':'entropy','random_state':1,'min_samples_leaf':15,'max_depth':6})
summary

{'fit': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=1,
             splitter='best'),
 'accuracy': 0.4539836037756161,
 'confusion_matrix': array([[ 56,  48,   0,   3,  24,   1],
        [ 16,  87,  11,  15,  27,   1],
        [ 15,  36,  14,  11,  32,   4],
        [  8,  24,   3,  15,  17,   1],
        [  6,  18,   5,  10, 154,  15],
        [  0,   3,   4,   1,  45,  10]], dtype=int64),
 'precision': 0.4059815069420106,
 'recall': 0.3705142601733085,
 'roc_auc': None,
 'leaf_nodes': 25,
 'non_leaf_nodes': 24,
 'size': 49,
 'time_taken': '0.08 seconds'}

In [461]:
dt_initial.saveTree('absenteeism')
dt_initial.fit.classes_

array(["'\\'(-inf-1.5]\\''", "'\\'(1.5-2.5]\\''", "'\\'(12-inf)\\''",
       "'\\'(2.5-3.5]\\''", "'\\'(3.5-7.5]\\''", "'\\'(7.5-12]\\''"],
      dtype=object)

In [462]:
summary['confusion_matrix']

array([[ 56,  48,   0,   3,  24,   1],
       [ 16,  87,  11,  15,  27,   1],
       [ 15,  36,  14,  11,  32,   4],
       [  8,  24,   3,  15,  17,   1],
       [  6,  18,   5,  10, 154,  15],
       [  0,   3,   4,   1,  45,  10]], dtype=int64)

# Experiments with One hot encoding for 'Reason for absence'

In [464]:
absenteeism_enc = oneHotEncodeRFA('Reason_for_absence',absenteeism)

In [465]:
predictors = absenteeism_enc.iloc[:,0:21]
response = absenteeism_enc.iloc[:,21]
dt_next = DecisionTreeModified(predictors,response)

In [466]:
summary = dt_next.trainClassifier({'criterion':'entropy','random_state':2,'max_features':10,'max_depth':5})
summary

{'fit': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
             max_features=10, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=2,
             splitter='best'),
 'accuracy': 0.32329011823513076,
 'confusion_matrix': array([[ 57,  31,  13,   2,  26,   3],
        [ 24,  51,  29,   6,  42,   5],
        [ 14,  44,  12,   4,  37,   1],
        [  7,  19,   6,  10,  24,   2],
        [ 20,  52,  18,   7, 106,   5],
        [ 11,  17,   6,   2,  24,   3]], dtype=int64),
 'precision': 0.283248019951287,
 'recall': 0.26134917650934886,
 'roc_auc': None,
 'leaf_nodes': 17,
 'non_leaf_nodes': 16,
 'size': 33,
 'time_taken': '0.06 seconds'}

In [14]:
dt_next.saveTree('absenteeism_enc')

# Experiment with Subset of features

In [493]:
absenteeism_sub = pd.read_csv(r'D:\WPI\DS-Coursework\CS548-KDD\Homework\HW2\dataset\Absenteeism_at_work_AAA\Absenteeism_python.csv')
absenteeism_sub = absenteeism_sub.drop(['Service_time','Education','Hit_target','Social_drinker','Social_smoker','Pet','Height','Body_mass_index'],1)
predictors = absenteeism_sub.iloc[:,0:9]
response = absenteeism_sub.iloc[:,9]
dt_subset = DecisionTreeModified(predictors,response)

In [494]:
summary = dt_subset.trainClassifier({'criterion':'entropy','random_state':2,'min_samples_leaf':10,'min_weight_fraction_leaf':0.02})
summary

{'fit': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=10, min_samples_split=2,
             min_weight_fraction_leaf=0.02, presort=False, random_state=2,
             splitter='best'),
 'accuracy': 0.42289222659394377,
 'confusion_matrix': array([[ 65,  36,   9,   4,  15,   3],
        [ 27,  65,  26,  18,  20,   1],
        [ 14,  31,  22,  15,  24,   6],
        [ 12,  15,  10,  15,  16,   0],
        [  6,  21,  20,   9, 134,  18],
        [  2,   0,   8,   1,  40,  12]], dtype=int64),
 'precision': 0.36907413657648896,
 'recall': 0.35969345795123236,
 'roc_auc': None,
 'leaf_nodes': 35,
 'non_leaf_nodes': 34,
 'size': 69,
 'time_taken': '0.07 seconds'}

In [495]:
dt_subset.saveTree('absenteeism_sub')

# Experiments with subset & One hot encoding for RFA

In [171]:
absenteeism_sub = absenteeism_enc
absenteeism_sub = absenteeism_sub.drop(['Service_time','Hit_target','Education','Social_drinker','Social_smoker','Pet','Height','Body_mass_index'],1)
predictors = absenteeism_sub.iloc[:,0:13]
response = absenteeism_sub.iloc[:,13]
dt_subenc = DecisionTreeModified(predictors,response)

In [207]:
summary = dt_subenc.trainClassifier({'criterion':'entropy','random_state':2,'max_features':10,'max_depth':5})
summary

{'fit': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=5,
             max_features=10, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=2,
             splitter='best'),
 'accuracy': 0.3711618473702209,
 'confusion_matrix': array([[ 48,  45,   8,   8,  21,   2],
        [ 13,  71,  14,   8,  46,   5],
        [ 14,  48,   5,   2,  41,   2],
        [ 12,  18,   3,   3,  31,   1],
        [  9,  39,   2,   8, 141,   9],
        [  2,  13,   3,   2,  36,   7]], dtype=int64),
 'leaf_nodes': 17,
 'non_leaf_nodes': 16,
 'size': 33}

In [209]:
dt_subenc.saveTree('absenteeism_subenc')

# Experiments with PCA

In [368]:
absenteeism = pd.read_csv(r'D:\WPI\DS-Coursework\CS548-KDD\Homework\HW2\dataset\Absenteeism_at_work_AAA\Absenteeism_python.csv')
predictors = absenteeism.iloc[:,0:17]
response = absenteeism.iloc[:,17]
pca_initial = PCAModified(predictors)
absenteeism_pca = pca_initial.reduceDimensions()

In [369]:
absenteeism_pca_df = pd.DataFrame(data=absenteeism_pca)

In [370]:
dt_pca = DecisionTreeModified(absenteeism_pca_df,response)

In [376]:
summary = dt_pca.trainClassifier({'criterion':'entropy','random_state':2,'min_samples_leaf':15,'max_depth':10})
summary

{'fit': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=10,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=2,
             splitter='best'),
 'accuracy': 0.2949166537713711,
 'confusion_matrix': array([[30, 28, 15,  5, 44, 10],
        [25, 60, 14, 20, 34,  4],
        [17, 32, 15, 15, 24,  9],
        [10, 17,  9, 11, 20,  1],
        [35, 42,  9, 14, 89, 19],
        [ 7,  5,  5,  7, 26, 13]], dtype=int64),
 'leaf_nodes': 36,
 'non_leaf_nodes': 35,
 'size': 71}

In [233]:
summary['confusion_matrix']

array([[ 28,  26,  12,   4,  59,   3],
       [ 13,  62,  17,  16,  48,   1],
       [ 18,  26,  20,  11,  35,   2],
       [  6,  17,   7,  11,  27,   0],
       [ 29,  44,   6,  18, 109,   2],
       [  7,   6,   4,   8,  36,   2]], dtype=int64)

In [234]:
dt_pca.saveTree('absenteeism_pca')

# Experiments with different discretization of target

In [507]:
absenteeism = pd.read_csv(r'D:\WPI\DS-Coursework\CS548-KDD\Homework\HW2\dataset\Absenteeism_at_work_AAA\Absenteeism_lvl0.csv')
predictors = absenteeism.iloc[:,0:17]
response = absenteeism.iloc[:,17]
response = discretizeAttr(response,4)

In [508]:
response.unique()

array(['(3.0, 8.0]', '(-0.001, 2.0]', '(8.0, 120.0]', '(2.0, 3.0]'],
      dtype=object)

In [509]:
dt_disc = DecisionTreeModified(predictors,response)

In [513]:
summary = dt_disc.trainClassifier({'criterion':'entropy','random_state':2,'min_samples_leaf':7},['(-0.001, 1.0]','(1.0, 2.0]','(2.0, 3.0]','(3.0, 8.0]','(8.0, 120.0]'])
summary

{'fit': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=7, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=2,
             splitter='best'),
 'accuracy': 0.5223304061090651,
 'confusion_matrix': array([[  0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0],
        [  0,   0,  23,  46,   4],
        [  0,   0,  27, 159,  28],
        [  0,   0,   7,  31,  14]], dtype=int64),
 'precision': 0.4254106330004327,
 'recall': 0.4161414938950858,
 'roc_auc': None,
 'leaf_nodes': 73,
 'non_leaf_nodes': 72,
 'size': 145,
 'time_taken': '0.09 seconds'}

In [500]:
dt_disc.saveTree('absenteeism_disc')

# Experiments after combining records of particular employee

In [288]:
absenteeism_lvl0 = pd.read_csv(r'D:\WPI\DS-Coursework\CS548-KDD\Homework\HW2\dataset\Absenteeism_at_work_AAA\Absenteeism_lvl0_0.csv')

In [338]:
n = int(max(absenteeism_lvl0['ID'])-1)
emp_list = []
for id in range(n+1):
    frame_ = absenteeism_lvl0[absenteeism_lvl0['ID']==id+1]
    transport = frame_['Transportation_expense'].mean()
    dfrw = frame_['Distance_from_Residence_to_Work'].mean()
    age = frame_['Age'].mean()
    wla = frame_['Work_load_Average/day_'].mean()
    dfail = frame_['Disciplinary_failure'].mean()
    edu = frame_['Education'].mean()
    child = frame_['Son'].mean()
    social_drinker = frame_['Social_drinker'].mean()
    social_smoker = frame_['Social_smoker'].mean()
    pet = frame_['Social_smoker'].mean()
    height = frame_['Height'].mean()
    bmi = frame_['Body_mass_index'].mean()
    abs_in_hr = frame_['Absenteeism_time_in_hours'].sum()
    emp_list.append([id+1,transport,dfrw,age,wla,dfail,edu,child,social_drinker,social_smoker,pet,height,bmi,abs_in_hr])

In [339]:
absenteeism_emp = pd.DataFrame(data=emp_list,columns=['id','Transport','Dfrw','age','wla','dfail','edu','child','social_drinker','social_smoker','pet','height','bmi','absenteeism'])

In [357]:
predictors = absenteeism_emp.iloc[:,0:13]
response = absenteeism_emp.iloc[:,13]
response = discretizeAttr(response,3)
predictors = predictors.drop('id',1)
response.unique()

array(['(30.667, 184.0]', '(-0.001, 30.667]', '(184.0, 482.0]'],
      dtype=object)

In [358]:
dt_emp = DecisionTreeModified(predictors,response)

In [361]:
summary = dt_emp.trainClassifier({'criterion':'entropy','random_state':2},labels=['(-0.001, 30.667]', '(30.667, 184.0]', '(184.0, 482.0]'])
summary

{'fit': DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=2,
             splitter='best'),
 'accuracy': 0.35,
 'confusion_matrix': array([[4, 5, 3],
        [7, 3, 2],
        [3, 3, 6]], dtype=int64),
 'leaf_nodes': 14,
 'non_leaf_nodes': 13,
 'size': 27}

In [362]:
dt_emp.saveTree('absenteeism_emp')